# Аттестационный проект на тему "Классификация названий лекарственных средств"

In [ ]:
!pip install stop_words 

In [ ]:
%matplotlib inline
import stop_words
import os
import time
import json
import torch
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import re
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
from sklearn.compose import ColumnTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer 
from sklearn.preprocessing import MultiLabelBinarizer 

In [ ]:
nltk.download('punkt')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df = pd.read_csv("utitles_cats_short.tsv", sep="\t")

In [ ]:
#pd.set_option('display.max_colwidth', None)
#df.head(5)

In [ ]:
df.dropna(how='any', axis=0)

In [ ]:
df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
df['Категория'] = df['Категория'].str.lower()

In [ ]:
df['Название'] = df['Название'].str.lower()

In [ ]:
df.Категория.value_counts()

In [ ]:
df.describe()

In [ ]:
russian_stopwords = get_stop_words('ru')
russian_stopwords.extend(['...', '«', '»', 'здравствуйте', 'здравствуй', 'до свидания', 'добрый день', 'добрый вечер', 'в', 'внимание', 'неопознанный', 'товар', 'яяя'])

def remove_punctuation(text):
    return ''.join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

prep_text = [remove_multiple_spaces(remove_punctuation(text.lower())) for text in df['Категория'].astype('str')]
df['Категория'] = prep_text

In [ ]:
prep_title = [remove_multiple_spaces(remove_punctuation(text.lower())) for text in df['Название'].astype('str')]
df['Название'] = prep_title

In [ ]:
data = df['Категория']

def tokenize_drugs(data):
    for drug in data:
        tokens = word_tokenize(drug)
        #print(tokens)

In [ ]:
balance_counts = df.groupby('Категория')['Категория'].agg('count').values
balance_counts

Попробуем подобрать параметры с помощью GridSearchCV

In [ ]:
from pprint import pprint
import logging

In [ ]:
texts = df['Название']
categories = df['Категория']

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train1, X_test1, y_train1, y_test1 = train_test_split(texts, categories, test_size=0.2, random_state=42)


In [ ]:
# Определение пайплайна
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

In [ ]:
# Определение параметров для Grid Search
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 1), (1, 2)),  # униграммы или биграммы
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
}

In [ ]:
# Поиск по сетке
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [ ]:
grid_search.fit(X_train1, y_train1)

In [ ]:
# Лучшие параметры и оценка модели
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
# Предсказание на тестовой выборке
y_pred1 = grid_search.predict(X_test1)

In [ ]:
# Оценка качества модели
accuracy = accuracy_score(y_test1, y_pred1)
precision = precision_score(y_test1, y_pred1, average='weighted')
recall = recall_score(y_test1, y_pred1, average='weighted')
f1 = f1_score(y_test1, y_pred1, average='weighted')

print(f"Точность (accuracy): {accuracy}")
print(f"Точность (precision): {precision}")
print(f"Полнота (recall): {recall}")
print(f"F-мера (f1 score): {f1}")

# TF-IDF

In [ ]:
# Создание экземпляра CountVectorizer с использованием собственной функции предварительной обработки
tfidf = TfidfVectorizer(norm=None, max_df=0.75, max_features=500, decode_error='replace') # 1) min_df=0.1 2)min_df=0.5

# Преобразование всех данных
X = tfidf.fit_transform(texts.values.tolist())

In [ ]:
y = df['Категория'] if 'Категория' in df.columns else [0] * len(df) 

In [ ]:
# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
LR_clf = LogisticRegression(random_state=64, solver='lbfgs', max_iter=20, n_jobs=-1) # Обучаем классификатор

In [ ]:
LR_clf.fit(X_train, y_train)

In [ ]:
LR_train_pred = LR_clf.predict(X_train)
LR_test_pred = LR_clf.predict(X_test)

In [ ]:
y_pred = LR_clf.predict(X_test)

In [ ]:
# Вывод результатов

accuracy_score(y_train, LR_train_pred), accuracy_score(y_test, LR_test_pred)
     
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
accuracy_score(y_train, LR_train_pred), accuracy_score(y_test, LR_test_pred)

In [ ]:
accuracy_tfidf = accuracy_score(y_test, y_pred)
precision_tfidf = precision_score(y_test, y_pred, average='weighted')
recall_tfidf = recall_score(y_test, y_pred, average='weighted')
f1_tfidf = f1_score(y_test, y_pred, average='weighted')

print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf, precision_tfidf, 
                                                                       recall_tfidf, f1_tfidf))

Наримуем схему пайплайна:

In [ ]:
!pip install graphviz

In [ ]:
from graphviz import Digraph

In [ ]:
# Создание объекта графа
dot = Digraph(comment='Text Classification Pipeline')

# Добавление узлов (элементов пайплайна)
dot.node('A', 'Raw Text Data')
dot.node('B', 'CountVectorizer')
dot.node('C', 'TfidfTransformer')
dot.node('D', 'SGDClassifier')

# Добавление рёбер (соединений между элементами)
dot.edge('A', 'B', label='vect__max_df: 1.0\nvect__ngram_range: (1, 2)')
dot.edge('B', 'C')
dot.edge('C', 'D', label='clf__alpha: 1e-05\nclf__max_iter: 20\nclf__penalty: elasticnet')

# Сохранение графа в файл
dot.render('text_classification_pipeline', format='png')

# Вывод графа
dot.view()

In [ ]:
# Визуализация пайплайна
import matplotlib.pyplot as plt
from sklearn import set_config
set_config(display='diagram')

grid_search.best_estimator_

# Word2Vec

In [ ]:
!pip install huggingface_hub

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
from huggingface_hub import hf_hub_download
from scipy.sparse import csr_matrix

In [ ]:

#model = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/wikipedia2vec_ruwiki_20180420_300d", filename="ruwiki_20180420_300d.txt"))


In [ ]:
# Создание словаря index_to_word
index_to_word = {index: word for word, index in tfidf.vocabulary_.items()}

In [ ]:
# Функция для преобразования разреженной матрицы обратно в текстовые данные
def sparse_matrix_to_texts(matrix, index_to_word):
    texts = []
    for row in matrix:
        words = [index_to_word[idx] for idx in row.indices for _ in range(int(row[0, idx]))]  # Учитываем частоты
        texts.append(' '.join(words))
    return texts

In [ ]:
# Преобразование разреженной матрицы X_train в текст
texts_train = sparse_matrix_to_texts(X_train, index_to_word)

In [ ]:
# Разделение текстов на слова для Word2Vec
sent = [text.split() for text in texts_train]

Сделаем первичную загрузку модели с huggingface:

In [ ]:
# Загрузка предобученной модели Word2Vec на 300
#model_path = hf_hub_download(repo_id="Word2vec/wikipedia2vec_ruwiki_20180420_300d", filename="ruwiki_20180420_300d.txt")
#pretrained_model = KeyedVectors.load_word2vec_format(model_path, binary=False)

In [ ]:
model_path = hf_hub_download(repo_id="Word2vec/wikipedia2vec_ruwiki_20180420_100d", filename="ruwiki_20180420_100d.txt")
pretrained_model = KeyedVectors.load_word2vec_format(model_path, binary=False)

Для повторной загрузки модели будем использовать следующую ячейку:  

In [ ]:
model_path = hf_hub_download(repo_id="Word2vec/wikipedia2vec_ruwiki_20180420_100d", filename="ruwiki_20180420_100d.txt")
pretrained_model = KeyedVectors.load_word2vec_format(model_path, binary=False)

In [ ]:
# Инициализация новой модели Word2Vec с использованием предобученных векторов, ДЕЛАЕМ ПЕРВЫЙ РАЗ
model = Word2Vec(vector_size=pretrained_model.vector_size, window=2, min_count=1, workers=2)

In [ ]:
#уменьшим размер данный для тестирования
small_sentences = sent[:1000]

In [ ]:
# Построение словаря из обучающих данных
model.build_vocab(small_sentences) #sent)

In [ ]:
# Дополнительное обучение модели на ваших данных, уменьшим число эпох с 30 до 10
model.train(small_sentences, total_examples=len(small_sentences), epochs=10, report_delay=1)

In [ ]:
# Загрузка предобученных векторов в модель
model.wv.vectors = pretrained_model.vectors
model.wv.index_to_key = pretrained_model.index_to_key
model.wv.key_to_index = pretrained_model.key_to_index

In [ ]:
#Получение вектора слова:

word_vector = model.wv['нурофен']  # Замените 'слово' на интересующее вас слово
print(word_vector)

In [ ]:
#Поиск похожих слов:

similar_words = model.wv.most_similar('пенталгин', topn=10)  # Замените 'слово' на интересующее вас слово
print(similar_words)

In [ ]:
#Проверка вектора:

if 'аспирин' in model.wv:
    print(f"Вектор для 'аспирин': {model.wv['аспирин']}")
else:
    print("Слово не найдено в модели.")

In [ ]:
model.save("word2vec_model_100.model")

In [ ]:
# Пусть путь к вашей сохраненной модели будет таким
model_path = "word2vec_model_100.model"

In [ ]:
#model = KeyedVectors.load("word2vec_model_100.model")

In [ ]:
batch_1 = df[:10000]

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(texts, categories, test_size=0.2, random_state=42)

In [ ]:
unknown_words = 0
total_words = 0

for sent in X_train_w2v[0]:
    for word in sent.split():
        total_words += 1
        if word not in model.wv:
            unknown_words += 1

print(f"Unknown words: {unknown_words}")
print(f"Total words: {total_words}")
print(f"Percentage of unknown words: {unknown_words / total_words * 100:.2f}%")

In [ ]:
#X_train_w2v представляет собой список или другую структуру данных,создадим датафрэйм
data = {'Название': X_train_w2v}  # Подставьте свои данные здесь

# Создаем DataFrame из данных
X_train_w2v_new = pd.DataFrame(data)

# Выводим первые несколько строк для проверки
print(X_train_w2v_new.head())

In [ ]:
def get_mean_w2v_vector(tokens, model, vector_size=100):  # Укажем размер вектора
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]

    if len(word_vectors) == 0:
        return np.zeros(vector_size)

    mean_vector = np.mean(word_vectors, axis=0)
    return mean_vector

In [ ]:
# Разделение текста на токены
X_train_w2v_new['tokens'] = X_train_w2v_new['Название'].apply(lambda x: x.split())

# Применение функции для получения средних векторов
X_train_w2v_new['vectors'] = X_train_w2v_new['tokens'].apply(lambda tokens: get_mean_w2v_vector(tokens, model))

In [ ]:
#X_train_w2v_new.head()

In [ ]:
# Создание DataFrame для X_test_w2v
data_test = {'Название': X_test_w2v}
X_test_w2v_new = pd.DataFrame(data_test)

In [ ]:
# Разделение текста на токены
X_test_w2v_new['tokens'] = X_test_w2v_new['Название'].apply(lambda x: x.split())

# Применение функции для получения средних векторов
X_test_w2v_new['vectors'] = X_test_w2v_new['tokens'].apply(lambda tokens: get_mean_w2v_vector(tokens, model))

In [ ]:
print(X_test_w2v_new.head())

In [ ]:
X_train_w2v_new['tokens'][5]

In [ ]:
print(f"Vocabulary size: {len(model.wv.index_to_key)}")

In [ ]:
HIDDEN = 100

In [ ]:
IdxTrain = [ix for ix, row in X_train_w2v_new.iterrows() if not isinstance(row['vectors'], np.ndarray) or np.all(row['vectors'] == 0)]
IdxTest = [ix for ix, row in X_test_w2v_new.iterrows() if not isinstance(row['vectors'], np.ndarray) or np.all(row['vectors'] == 0)]

In [ ]:
NewCols = ['col'+str(i) for i in range(HIDDEN)]
X_train_w2v_new[NewCols] = pd.DataFrame(X_train_w2v_new['vectors'].tolist(), index=X_train_w2v_new.index)
X_test_w2v_new[NewCols] = pd.DataFrame(X_test_w2v_new['vectors'].tolist(), index=X_test_w2v_new.index)

In [ ]:
# Проверка и сброс индексов в X_train_w2v_new
X_train_w2v_new = X_train_w2v_new.reset_index(drop=True)

# Проверка и сброс индексов в y_train_w2v
y_train_w2v = y_train_w2v.reset_index(drop=True)

In [ ]:
# Проверка и удаление несоответствующих строк
to_drop = [idx for idx in X_train_w2v_new.index if idx >= len(y_train_w2v)]
X_train_w2v_new = X_train_w2v_new.loc[~X_train_w2v_new.index.isin(to_drop)]
y_train_w2v = y_train_w2v.loc[~y_train_w2v.index.isin(to_drop)]

In [ ]:
# запуск модели Logistic Regression
lr_clf_w2v = LogisticRegression(random_state=64, solver='lbfgs', max_iter=500, n_jobs=-1) # Увеличение max_iter, чтобы избежать предупреждений о сходимости
lr_clf_w2v.fit(X_train_w2v_new[NewCols], y_train_w2v)

In [ ]:
# Предсказания на обучающем и тестовом наборах данных
lr_train_pred_w2v = lr_clf_w2v.predict(X_train_w2v_new[NewCols])
lr_test_pred_w2v = lr_clf_w2v.predict(X_test_w2v_new[NewCols])

In [ ]:
# Оценка точности на обучающем наборе данных
train_accuracy_w2v = accuracy_score(y_train_w2v, lr_train_pred_w2v)
print(f"Training Accuracy: {train_accuracy_w2v}")

In [ ]:
train_accuracy_w2v = accuracy_score(y_train_w2v, lr_train_pred_w2v)
test_accuracy_w2v = accuracy_score(y_test_w2v, lr_test_pred_w2v)

print(f"Training Accuracy: {train_accuracy_w2v}")
print(f"Testing Accuracy: {test_accuracy_w2v}")

# GloVe

Чтобы установить glove на Python необходмо:
Загрузить файл glove.6B.100d.txt с сайта GloVe или из другого репозитория.
Установить библиотеку gensim, используя команду pip install gensim.
Загрузить файл glove.6B.100d.txt в Python с помощью функции load_embeddings():

In [ ]:
!pip install gensim

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
glove_file = 'C:/Users/shevr/Documents/Project HSE/glove.6B.100d.txt'

In [ ]:
# Конвертируем файл формата GloVe в формат Word2Vec
glove2word2vec(glove_file, 'glove_word2vec.txt')

In [ ]:
# Загружаем модель GloVe с помощью библиотеки gensim
glove_model = KeyedVectors.load_word2vec_format('glove_word2vec.txt')

Создадим словарь уникальных слов в нашем датасете.
Разделим текст на токены и преобразуем их в список индексов словаря.
Используем метод similarity() класса KeyedVectors для вычисления сходства между текстом и каждым классом в вашей задаче классификации.
Выбераем класс с наибольшим значением сходства как предсказанный класс для данного текста.

In [ ]:
def get_mean_glove_vector(tokens, glove_model, vector_size=100):
    vectors = [glove_model[word] for word in tokens if word in glove_model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(vector_size)
    return mean_vector

# Пример использования
X_train_w2v_new['glove_vectors'] = X_train_w2v_new['tokens'].apply(lambda tokens: get_mean_glove_vector(tokens, glove_model))

print(X_train_w2v_new.head())

In [ ]:
# Список уникальных категорий
unique_classes = df['Категория'].unique()

# Разделим категории на отдельные слова
class_words = [cls.split() for cls in unique_classes]

# Проверим, какие слова есть в модели GloVe
present_words = [word for sublist in class_words for word in sublist if word in glove_model]
missing_words = [word for sublist in class_words for word in sublist if word not in glove_model]

print(f"Present words in GloVe: {present_words}")
print(f"Missing words in GloVe: {missing_words}")

In [ ]:
# Создание векторного представления для категорий
class_vectors = {}
for cls in unique_classes:
    words = cls.split()
    class_vectors[cls] = get_mean_glove_vector(words, glove_model)

print(f"Class vectors: {class_vectors}")

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
def glove_classification(text_title, glove_model, class_vectors):
    tokens = word_tokenize(text_title)  # Преобразование названия в токены
    mean_vector = get_mean_glove_vector(tokens, glove_model)
    
    similarities = {}
    for cls, vec in class_vectors.items():
        similarities[cls] = 1 - cosine(mean_vector, vec)  # 1 - косинусное расстояние для сходства
    
    predicted_class = max(similarities, key=similarities.get)
    return predicted_class

Этот код принимает на вход текст и список классов, а возвращает предсказанный класс для данного текста с использованием модели GloVe.

In [ ]:
# Пример использования
text_title = 'алмагель'
predicted_class = glove_classification(text_title, glove_model, class_vectors)
print(f"Predicted class: {predicted_class}")

На основе моих данных нарисую пошаговую схему проекта

In [ ]:
from graphviz import Digraph

# Создание объекта графа
dot = Digraph(comment='Классификация названий лекарственных средств')

# Добавление узлов (элементов пайплайна)
dot.node('A', 'Сбор данных')
dot.node('B', 'Предобработка текста')
dot.node('C', 'Векторизация')
dot.node('D', 'Разделение данных')
dot.node('E', 'Обучение модели')
dot.node('F', 'Оценка модели')
dot.node('G', 'Применение модели')

# Добавление рёбер (соединений между элементами)
dot.edge('A', 'B', label='Очистка данных')
dot.edge('B', 'C', label='TF-IDF, Word2Vec, GloVe')
dot.edge('C', 'D', label='Обучающая и тестовая выборки')
dot.edge('D', 'E', label='Логистическая регрессия, SGDClassifier')
dot.edge('E', 'F', label='Точность, полнота, F-мера')
dot.edge('F', 'G', label='Классификация новых данных')

# Сохранение графа в файл
dot.render('drug_classification_pipeline', format='png')

# Отображение графа
dot.view()